In [1]:
import os
import sys
import functools

sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2018,22)
from grid import Grid

In [2]:
# map area_types
area_type = {
    0: '.',
    1: '=',
    2: '|'
}

# reverse map to risk_level
risk_level =  {v: k for k, v in area_type.items()}

def parse_input(lines):
    depth = int(lines[0].split(' ')[1])
    target = lines[1].split(' ')[1]
    x_pos, y_pos = [int(pos) for pos in target.split(',')]
    return depth, (x_pos, y_pos)

@functools.lru_cache(maxsize=None)
def get_geologic_index(pos, depth, target):
    # print(f"get_geologic_index({pos}, {depth}, {target}):")
    # The region at 0,0 (the mouth of the cave) has a geologic index of 0.
    # The region at the coordinates of the target has a geologic index of 0.
    if pos in [(0,0), target]:
        return 0
    if pos[1] == 0:
        # If the region's Y coordinate is 0, the geologic index is its X coordinate times 16807.
        return pos[0] * 16807
    # If the region's X coordinate is 0, the geologic index is its Y coordinate times 48271.
    if pos[0] == 0:
        return pos[1] * 48271
    # Otherwise, the region's geologic index is the result of multiplying the erosion levels of the regions at X-1,Y and X,Y-1.
    return get_erosion_level((pos[0] - 1, pos[1]), depth, target) * get_erosion_level((pos[0], pos[1] - 1), depth, target)

@functools.lru_cache(maxsize=None)
def get_erosion_level(pos, depth, target):
    """
    Function to calculate erosion level for an area
    Args:
        pos: tuple() -  int(x), int(y)
        depth: int()
        target: tuple() -  int(x), int(y)
    
    Returns:
        erosion_level: int()
    """
    # A region's erosion level is its geologic index plus the cave system's depth, all modulo 20183.
    geologic_index = get_geologic_index(pos, depth, target)
    return (geologic_index + depth) % 20183

@functools.lru_cache(maxsize=None)
def get_area_type(pos, depth, target):
    """
    function to get area type
    Args:
        pos: tuple() -  int(x), int(y)
        depth: int()
        target: tuple() -  int(x), int(y)
    Returns:
        area_type: char() '.', '=', or '|'
    """
    # If the erosion level modulo 3 is 0, the region's type is rocky.
    # If the erosion level modulo 3 is 1, the region's type is wet.
    # If the erosion level modulo 3 is 2, the region's type is narrow.
    erosion_level = get_erosion_level(pos, depth, target)
    return area_type[erosion_level % 3]

def get_risk_level(grid, start, goal):
    """
    Function to calculate risk level
    Args:
        grid: Grid() current map
        start: tuple() start position x/y coordinates
        goal: tuple() end position x/y coordinates
    Returns:
        total: int() risk_level total
    """
    total = 0
    min_x = min(start[0], goal[0])
    min_y = min(start[1], goal[1])
    max_x = max(start[0], goal[0])
    max_y = max(start[1], goal[1])    
    for point in grid:
        if min_x <= point[0] <= max_x and min_y <= point[1] <= max_y:
            total += risk_level[grid.get_point(point)]
    return total


In [3]:
input_text = """depth: 510
target: 10,10"""
input_text = """depth: 4845
target: 6,770"""
my_grid = Grid('M', type="infinite", coordinate_system='screen', use_overrides=False)
print(my_grid)
depth, target = parse_input(input_text.splitlines())
print("test Cases:")
for point in [(0,0), (1,0), (0,1), (1,1), (10, 10)]:
    my_area_type = get_area_type(point, depth, target)
    print(f"{point}: {my_area_type}")



M
test Cases:
(0, 0): .
(1, 0): |
(0, 1): .
(1, 1): =
(10, 10): .


In [4]:
print(depth, target)
scale_out = 100
for x_pos in range(target[0] + scale_out):
    for y_pos in range(target[1] + scale_out):
        my_grid.set_point((x_pos, y_pos), get_area_type((x_pos, y_pos), depth, target))
my_grid.update()

risk_level = get_risk_level(my_grid, (0,0), target)
# my_grid.cfg['use_overrides'] = True
# my_grid.overrides = {
#     (0,0): 'M',
#     target: 'T'
# }
# print(my_grid)
print(f"Risk Level is {risk_level}")

4845 (6, 770)
Risk Level is 5400


In [5]:
from queue import PriorityQueue
from grid import manhattan_distance, Node

class ExtendedNode(Node):
    def __init__(self, *args, **kwargs):
        kwargs['skip_scoring'] = True
        super().__init__(*args, **kwargs)
        # default to torch for starting
        self.equipped = kwargs.get('equipped', 'torch')
        # get area_type from map, or calculate it if we have gone off grid
        self.area_type = self.grid.get_point(self.position)
        if self.area_type not in '.=|':
            self.area_type = get_area_type(self.position, depth, target)
        self.g_score = self.calc_g_score()
        self.h_score = self.calc_h_score()
        self.f_score = self.calc_f_score()
    
    def recalc_scores(self):
        if self.parent:
            self.parent.recalc_scores()
        self.g_score = self.calc_g_score()
        self.h_score = self.calc_h_score()
        self.f_score = self.calc_f_score()
    
    def calc_g_score(self):
        if not self.parent:
            return 0
        # Moving to an adjacent region takes one minute
        self.g_score = self.parent.g_score + 1
        # Switching to using the climbing gear, torch, or neither always takes seven minutes
        if self.equipped != self.parent.equipped:
            self.g_score += 7
        # Finally, once you reach the target, you need the torch equipped before you can find him in the dark.
        # The target is always in a rocky region, so if you arrive there with climbing gear equipped, you will
        # need to spend seven minutes switching to your torch.
        if self.position == self.goal and self.equipped != 'torch':
            self.g_score += 7
        return self.g_score
    
    def calc_f_score(self):
        return self.g_score + self.h_score
        # return float(f"{self.g_score}.{self.h_score}")
    
    def get_children(self, child_pos):
        # In rocky regions, you can use the climbing gear or the torch. You cannot use neither (you'll likely slip and fall).
        # In wet regions, you can use the climbing gear or neither tool. You cannot use the torch (if it gets wet, you won't have a light source).
        # In narrow regions, you can use the torch or neither tool. You cannot use the climbing gear (it's too bulky to fit).
        if min(child_pos) < 0:
            return []
        
        # detect loop
        if self.find_node_by_position(child_pos):
            return []
        equipment_terrain_map = {
            'climbing_gear': '.=',
            'torch': '.|',
            'neither': '=|'
        }

        terrain_equipment_map = {
            '.': ['climbing_gear', 'torch'],
            '=': ['climbing_gear', 'neither'],
            '|': ['neither', 'torch']
        }
        children  = []
        child_area_type = get_area_type(child_pos, depth, target)
        # equipment parent can use
        for equipment in terrain_equipment_map[self.area_type]:
            if equipment in terrain_equipment_map[child_area_type]:
                children.append(
                    self.__class__(
                        child_pos,
                        self.goal,
                        self,
                        grid=self.grid,
                        equipped=equipment
                    )
                )
        # if self.position[1] == 4:
        #     print(f"At {self.position}:")
        #     print(f"    self: {self.area_type} - {self.equipped} , possible: {terrain_equipment_map[self.area_type]}")
        #     print(f"    child: {child_pos} - {child_area_type} , possible: {terrain_equipment_map[child_area_type]}")        
        #     for child in children:
        #         print(f"child: {child}")
        return children
    
    def find_node_by_position(self, target):
        node = self
        while node.position != target:
            node = node.parent
            if not node:
                return None
        return node
    
    def __eq__(self, other):
        """
        Node equal
        """
        return self.position == other.position and self.equipped == other.equipped
    
    def __hash__(self):
        """
        Node Hash
        """
        return hash((self.position, self.equipped))

    def __str__(self):
        return f"""pos: {self.position}, type: {self.area_type}, equip: {self.equipped}, g_score: {self.g_score}"""
        
       
        
# # find first shortest path
# nodes = my_grid.shortest_paths((0,0), target, node_class=ExtendedNode, retval='nodes', max_paths=1)
# # The closed set in our A* algorithm can block shorter paths when there are multiple paths running
# # through the same point.
# # so lets make sure the path is optimal

# node = nodes[0]
# path = node.path()
# print(node.g_score)
# print(path)

# # for idx, start_node in enumerate(nodes):
# #     print(f"path {idx}:")
# #     node = start_node
# #     while node:
# #         print(node)
# #         node = node.parent

In [6]:
my_grid.update()
# print(my_grid)
import networkx

# init graph
graph = networkx.Graph()
# start with torch at (0, 0)
terrain_equipment_map = {
    '.': ['climbing_gear', 'torch'],
    '=': ['climbing_gear', 'neither'],
    '|': ['neither', 'torch']
}
for point in my_grid:
    point_terrain = my_grid.get_point(point)
    for eq_1 in terrain_equipment_map[point_terrain]:
        for eq_2 in terrain_equipment_map[point_terrain]:
            if  eq_1 == eq_2:
                continue
            # print(f"Adding edge: {(point[0], point[1], eq_1)} => {(point[0], point[1], eq_2)}")
            graph.add_edge((point[0], point[1], eq_1), (point[0], point[1], eq_2), weight=7)
for point in my_grid:
    point_terrain = my_grid.get_point(point)
    for direction, neighbor in my_grid.get_neighbors(point=point, directions=['n','e','s','w']).items():
        if direction not in ['n','e','s','w']:
            continue
        for equipped in terrain_equipment_map[point_terrain]:
            if graph.has_node((neighbor[0], neighbor[1], equipped)):
                graph.add_edge((point[0], point[1], equipped), (neighbor[0], neighbor[1], equipped), weight=1)

print("Number of nodes:", graph.number_of_nodes())
print("Number of edges:", graph.number_of_edges())

answer = networkx.dijkstra_path_length(graph, (0, 0, 'torch'), (target[0], target[1], 'torch'))
print(f"{(0,0)} to {target} = {answer}")

Number of nodes: 184440
Number of edges: 337077
(0, 0) to (6, 770) = 1048


In [7]:
for node in graph.nodes():
    print(node)
    break

(0, 0, 'climbing_gear')


In [8]:
networkx.dijkstra_path(graph, (0, 0, 'torch'), (target[0], target[1], 'torch'))

[(0, 0, 'torch'),
 (1, 0, 'torch'),
 (2, 0, 'torch'),
 (3, 0, 'torch'),
 (3, 1, 'torch'),
 (4, 1, 'torch'),
 (4, 2, 'torch'),
 (4, 3, 'torch'),
 (4, 4, 'torch'),
 (4, 4, 'climbing_gear'),
 (4, 5, 'climbing_gear'),
 (4, 6, 'climbing_gear'),
 (4, 7, 'climbing_gear'),
 (4, 8, 'climbing_gear'),
 (4, 9, 'climbing_gear'),
 (4, 10, 'climbing_gear'),
 (4, 11, 'climbing_gear'),
 (5, 11, 'climbing_gear'),
 (5, 12, 'climbing_gear'),
 (5, 13, 'climbing_gear'),
 (5, 14, 'climbing_gear'),
 (5, 15, 'climbing_gear'),
 (5, 16, 'climbing_gear'),
 (5, 17, 'climbing_gear'),
 (5, 18, 'climbing_gear'),
 (5, 19, 'climbing_gear'),
 (5, 20, 'climbing_gear'),
 (5, 21, 'climbing_gear'),
 (5, 22, 'climbing_gear'),
 (5, 23, 'climbing_gear'),
 (6, 23, 'climbing_gear'),
 (6, 24, 'climbing_gear'),
 (7, 24, 'climbing_gear'),
 (8, 24, 'climbing_gear'),
 (8, 25, 'climbing_gear'),
 (8, 26, 'climbing_gear'),
 (8, 27, 'climbing_gear'),
 (9, 27, 'climbing_gear'),
 (9, 28, 'climbing_gear'),
 (9, 29, 'climbing_gear'),
 (9, 30

In [9]:
for node in graph.nodes:
    if node[0] == 0 and node[1] == 0:
        print(node)

(0, 0, 'climbing_gear')
(0, 0, 'torch')
